Show an example of how rolling cadence changes the distribution of visits

In [2]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import healpy as hp
import lsst.sims.maf.db as db
import lsst.sims.maf.utils as utils
import lsst.sims.maf.metrics as metrics
import lsst.sims.maf.slicers as slicers
import lsst.sims.maf.metricBundles as metricBundles

In [3]:
outDir ='Rolling'
dbFile = 'ops2_1098_sqlite.db'
opsimdb = utils.connectOpsimDb(dbFile)
resultsDb = db.ResultsDb(outDir=outDir)
nightLimits = (np.arange(3)+1)*365
nightLimits = np.append(nightLimits, 3653.)
nside = 64

IOError: Sqlite database file "ops2_1098_sqlite.db" not found.

In [ ]:
# parameters for SNe
peaks = {'uPeak':25.9, 'gPeak':23.6, 'rPeak':22.6, 'iPeak':22.7, 'zPeak':22.7,'yPeak':22.8}
peakTime = 15
transDuration = peakTime+30 # Days

In [ ]:
bundleList = []
plotDict ={}
pdL = [{'colorMin':0,'colorMax':100}, {'colorMin':40,'colorMax':100},
       {'colorMin':40,'colorMax':120},{'colorMin':200,'colorMax':400}]
sm = metrics.MedianMetric()
runName='ops2_1098'
for night,pd in zip(nightLimits,pdL):
    metric = metrics.CountMetric(col='expMJD')
    slicer = slicers.HealpixSlicer(nside=nside,lonCol='ditheredRA',latCol='ditheredDec')
    sql = 'filter = "r" and night < %i' % night
    bundleList.append(metricBundles.MetricBundle(metric,slicer,sql, plotDict=pd, 
                                                 summaryMetrics=sm, runName=runName))
    metric = metrics.InterNightGapsMetric(metricName='Median Inter-Night Gap')
    bundleList.append(metricBundles.MetricBundle(metric,slicer,sql, runName=runName,
                                                 plotDict={'colorMin':0,'colorMax':30, 'xMin':0,'xMax':30}))

    metric = metrics.TransientMetric(riseSlope= -2./peakTime, declineSlope=1.4/30., 
                                      transDuration=transDuration, peakTime=peakTime, surveyDuration=night/365.25, 
                                      nFilters=3, nPrePeak=3, nPerLC=2, **peaks)
    sql = '(filter="r" or filter="g" or filter="i" or filter="z") and night < %i' % (night)
    bundleList.append(metricBundles.MetricBundle(metric,slicer,sql, runName=runName,
                                                 plotDict={})

In [ ]:
bd = metricBundles.makeBundlesDictFromList(bundleList)
bg =  metricBundles.MetricBundleGroup(bd, opsimdb,
                                      outDir=outDir, resultsDb=resultsDb)

In [4]:
bg.runAll()
bg.plotAll(closefigs=False)

NameError: name 'bg' is not defined

In [ ]:
for bundle in bundleList:
    if len(bundle.summaryValues) > 0:
        print bundle.sqlconstraint, bundle.summaryValues

In [ ]:
dbFile = 'enigma_1260_sqlite.db' # Cheese2
opsimdb = utils.connectOpsimDb(dbFile)
bundleList = []
plotDict ={}
sm = metrics.MedianMetric()
runName='enigma_1260'
for night,pd in zip(nightLimits,pdL):
    metric = metrics.CountMetric(col='expMJD')
    slicer = slicers.HealpixSlicer(nside=nside,lonCol='ditheredRA',latCol='ditheredDec')
    sql = 'filter = "r" and night < %i' % night
    bundleList.append(metricBundles.MetricBundle(metric,slicer,sql, plotDict=pd, 
                                                 runName=runName, summaryMetrics=sm))
    metric = metrics.InterNightGapsMetric(metricName='Median Inter-Night Gap')
    bundleList.append(metricBundles.MetricBundle(metric,slicer,sql, runName=runName,
                                                 plotDict={'colorMin':0,'colorMax':30, 'xMin':0,'xMax':30}))
bd = metricBundles.makeBundlesDictFromList(bundleList)
bg =  metricBundles.MetricBundleGroup(bd, opsimdb,
                                      outDir=outDir, resultsDb=resultsDb)

In [5]:
bg.runAll()
bg.plotAll(closefigs=False)

NameError: name 'bg' is not defined

In [ ]:
for bundle in bundleList:
    if len(bundle.summaryValues) > 0:
        print bundle.sqlconstraint, bundle.summaryValues